In [1]:
import MetaTrader5 as mt5
from dotenv import load_dotenv
import os
import csv

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("EE_MT5_LOGIN"))
password = os.getenv("EE_MT5_PASSWORD")
server = os.getenv("EE_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# Fetch all symbols currently visible in Market Watch
symbols = mt5.symbols_get()

# Filter only active tradable symbols
active_symbols = [
    sym.name
    for sym in symbols
    if sym.visible and sym.trade_mode == mt5.SYMBOL_TRADE_MODE_FULL
]

# Print active symbols
print("Active symbols in Market Watch:")
for name in active_symbols:
    print(name)

# -----------------------------
# Export to CSV
# -----------------------------
csv_filename = "active_symbols.csv"

with open(csv_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Symbol"])  # header
    for name in active_symbols:
        writer.writerow([name])

print(f"\nExported {len(active_symbols)} symbols to {csv_filename}")

# Shutdown MT5 connection
mt5.shutdown()

Active symbols in Market Watch:
EURUSD
AUDUSD
GBPUSD
NZDUSD
USDCAD
USDCHF
USDJPY
DE30
FRA40
JPN225
SPX500
UK100
US100
US30
XAGUSD
XAUUSD
WTI

Exported 17 symbols to active_symbols.csv


True

### Max Sharpe Weights

In [14]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from scipy.optimize import minimize
from dotenv import load_dotenv
import os

# -----------------------------
# Setup & MT5 initialization
# -----------------------------
load_dotenv()
login = int(os.getenv("EE_MT5_LOGIN"))
password = os.getenv("EE_MT5_PASSWORD")
server = os.getenv("EE_MT5_SERVER")

if not mt5.initialize(login=login, password=password, server=server):
    print("initialize() failed, error code =", mt5.last_error())
    quit()

# -----------------------------
# Config
# -----------------------------
risk_free_rate = 0.0  # annual risk-free rate

# FX pairs needed to convert index returns to USD
fx_map = {
    "GER30.pro": "EURUSD.pro",
    "JPN225.pro": "USDJPY.pro",   # flip sign
    "HK50.pro": "USDHKD.pro",     # flip sign
    "UK100.pro": "GBPUSD.pro",
    "EUSTX50.pro": "EURUSD.pro"
}

# -----------------------------
# Helpers
# -----------------------------
def fetch_mt5_data(symbol, days=60):
    utc_to = pd.Timestamp.now()
    utc_from = utc_to - pd.Timedelta(days=days)

    rates = mt5.copy_rates_range(symbol, mt5.TIMEFRAME_D1, utc_from, utc_to)
    if rates is None:
        raise ValueError(f"Failed to fetch data for {symbol}")

    df = pd.DataFrame(rates)
    df["time"] = pd.to_datetime(df["time"], unit="s")
    df.set_index("time", inplace=True)
    return df


def calculate_log_returns(prices: pd.Series) -> pd.Series:
    return np.log(prices / prices.shift(1)).dropna()

# -----------------------------
# Load expected returns
# -----------------------------
# ai_factor_signal_acg.csv:  asset \t factor_signal
expected_returns_df = pd.read_csv("ai_factor_signal_ee.csv", sep="\t", header=None)
expected_returns_df.columns = ["asset", "factor_signal"]

# -----------------------------
# Z-score the factor signals
# -----------------------------
mean_signal = expected_returns_df["factor_signal"].mean()
std_signal = expected_returns_df["factor_signal"].std()

expected_returns_df["zscore_signal"] = (expected_returns_df["factor_signal"] - mean_signal) / std_signal

# Convert to dict
expected_returns_dict = dict(zip(expected_returns_df["asset"], expected_returns_df["zscore_signal"]))

symbols = list(expected_returns_dict.keys())

# -----------------------------
# Fetch price data for assets
# -----------------------------
data = {symbol: fetch_mt5_data(symbol) for symbol in symbols}

# -----------------------------
# Fetch FX data and compute FX log returns
# -----------------------------
fx_symbols = set(fx_map[s] for s in symbols if s in fx_map)
fx_data = {fx: fetch_mt5_data(fx) for fx in fx_symbols}

fx_returns = {}
for fx, df_fx in fx_data.items():
    lr = calculate_log_returns(df_fx["close"])

    # If pair is quoted as USDXXX (e.g., USDJPY, USDHKD), flip sign to get XXXUSD log return
    if fx.startswith("USD"):
        lr = -lr  # log(1/x) = -log(x)

    fx_returns[fx] = lr

# -----------------------------
# Compute USD-adjusted log returns for all assets
# -----------------------------
asset_returns = {}

for symbol in symbols:
    idx_lr = calculate_log_returns(data[symbol]["close"])

    if symbol in fx_map:
        fx_symbol = fx_map[symbol]
        fx_lr = fx_returns[fx_symbol]

        # Align index and FX by date
        combined = pd.concat([idx_lr, fx_lr], axis=1, join="inner")
        combined.columns = ["idx", "fx"]

        # USD-adjusted return: r_usd = r_index_local + r_fx
        asset_returns[symbol] = combined["idx"] + combined["fx"]
    else:
        # Already in USD terms
        asset_returns[symbol] = idx_lr

# -----------------------------
# Build aligned returns matrix
# -----------------------------
returns = pd.DataFrame(asset_returns).dropna(how="any")

# -----------------------------
# Covariance matrix (daily, log returns)
# -----------------------------
cov_matrix = returns.cov()

# Expected daily returns vector (assumed to be in USD terms already)
expected_returns = np.array([expected_returns_dict[symbol] for symbol in symbols])
expected_returns = expected_returns * returns.std().values  # scale by asset volatilities

# -----------------------------
# Max Sharpe optimization
# -----------------------------
def max_sharpe_ratio(expected_returns, cov_matrix, risk_free_rate=0.0):
    num_assets = len(expected_returns)

    def neg_sharpe(weights):
        port_ret = np.dot(weights, expected_returns)
        port_vol = np.sqrt(weights.T @ cov_matrix @ weights)
        return -(port_ret - risk_free_rate) / port_vol

    # Dollar-neutral: sum(weights) = 0
    constraints = [{"type": "eq", "fun": lambda w: np.sum(w)}]

    # Allow long/short
    bounds = [(-1, 1)] * num_assets

    # Start with zero‑sum initial guess
    init_guess = np.ones(num_assets)
    init_guess[: num_assets // 2] = 1
    init_guess[num_assets // 2 :] = -1
    init_guess = init_guess / np.sum(np.abs(init_guess))

    result = minimize(neg_sharpe, init_guess, bounds=bounds, constraints=constraints)

    if not result.success:
        raise ValueError(f"Optimization failed: {result.message}")

    weights = result.x
    daily_ret = np.dot(weights, expected_returns)
    daily_vol = np.sqrt(weights.T @ cov_matrix @ weights)

    return weights, daily_ret, daily_vol

# -----------------------------
# Run optimization
# -----------------------------
weights, daily_port_return, daily_port_vol = max_sharpe_ratio(
    expected_returns, cov_matrix, risk_free_rate=risk_free_rate
)

# -----------------------------
# Export optimal weights
# -----------------------------
weights_df = pd.DataFrame({
    "Symbol": symbols,
    "Weight": weights
})
weights_df.to_csv("ee_optimal_portfolio_weights.csv", header=False, index=False)
print("\nWeights exported to ee_optimal_portfolio_weights.csv")

# -----------------------------
# Annualized statistics (using daily log returns)
# -----------------------------
annual_port_return = (1 + daily_port_return)**252 - 1
annual_port_vol = daily_port_vol * np.sqrt(252)
annual_sharpe = (annual_port_return - risk_free_rate) / annual_port_vol

# -----------------------------
# Output
# -----------------------------
print("\nOptimal Portfolio Weights:")
for symbol, weight in zip(symbols, weights):
    print(f"{symbol}: {weight:.4f}")

print("\n--- Portfolio Statistics ---")
print(f"Daily Portfolio Return: {daily_port_return:.4%}")
print(f"Annualized Portfolio Return: {annual_port_return:.4%}")
print(f"Daily Portfolio Volatility: {daily_port_vol:.4%}")
print(f"Annualized Portfolio Volatility: {annual_port_vol:.4%}")
print(f"Annualized Sharpe Ratio: {annual_sharpe:.2f}")

# -----------------------------
# Shutdown MT5
# -----------------------------
mt5.shutdown()


Weights exported to ee_optimal_portfolio_weights.csv

Optimal Portfolio Weights:
AUDUSD: -0.8096
EURUSD: 0.5580
GBPUSD: 0.7759
NZDUSD: 0.0053
USDJPY: -0.3874
USDCHF: 0.7231
USDCAD: -0.9783
XAGUSD: -0.0954
XAUUSD: 0.2910
WTI: -0.2990
SPX500: -0.4739
US100: 0.3022
US30: 0.0003
DE30: -0.0264
JPN225: 0.2334
UK100: 0.9597
FRA40: -0.7791

--- Portfolio Statistics ---
Daily Portfolio Return: 1.1293%
Annualized Portfolio Return: 1594.2897%
Daily Portfolio Volatility: 0.3446%
Annualized Portfolio Volatility: 5.4704%
Annualized Sharpe Ratio: 291.44


True

### Volatility Targeted Weights using EWMA estimates

In [17]:
import pandas as pd
import numpy as np

# --- Load weights from comma-separated file ---
weights_df = pd.read_csv(
    "ee_optimal_portfolio_weights.csv",
    sep=",",
    header=None,
    engine="python",
    encoding="utf-8-sig"
)

weights_df.columns = ["asset", "weight"]

# Align weights to returns.columns order
weights_df = weights_df.set_index("asset").reindex(returns.columns)

# Extract weight vector
weights = weights_df["weight"].values

print("Loaded weights (aligned):")
print(weights)
print("\nAsset order:")
print(list(returns.columns))

# ============================================================
# --- EWMA COVARIANCE MATRIX ---
# ============================================================

lambda_ = 0.94  # RiskMetrics default
rets = returns.values
n = rets.shape[1]

# Start with sample covariance as initial value
ewma_cov = returns.cov().values.copy()

# Iterate through returns to update EWMA covariance
for t in range(1, len(rets)):
    r = rets[t-1].reshape(-1, 1)
    ewma_cov = lambda_ * ewma_cov + (1 - lambda_) * (r @ r.T)

# Portfolio EWMA volatility
ewma_daily_vol = np.sqrt(weights.T @ ewma_cov @ weights)
ewma_annual_vol = ewma_daily_vol * np.sqrt(252)
daily_returns = returns @ weights
annual_return = daily_returns.mean() * 252

print("\n--- EWMA VOLATILITY FORECAST ---")
print(f"EWMA Daily Volatility: {ewma_daily_vol:0.2%}")
print(f"EWMA Annual Volatility: {ewma_annual_vol:0.2%}")
print(f"EWMA Daily Portfolio Return: {daily_returns.mean():0.2%}")
print(f"EWMA Annual Portfolio Return: {annual_return:0.2%}")

# ============================================================
# --- SCALE PORTFOLIO TO TARGET VOLATILITY USING EWMA ---
# ============================================================

target_vol = 0.10  # 10% annual volatility

scale_factor = target_vol / ewma_annual_vol
scaled_weights = weights * scale_factor

# Recompute volatility after scaling
scaled_daily_vol = np.sqrt(scaled_weights.T @ ewma_cov @ scaled_weights)
scaled_annual_vol = scaled_daily_vol * np.sqrt(252)

# Compute scaled returns
scaled_daily_returns = returns @ scaled_weights
scaled_avg_daily_return = scaled_daily_returns.mean()
scaled_annual_return = scaled_avg_daily_return * 252

print(f"\n--- SCALED PORTFOLIO (Target Vol = {target_vol:.0%}) ---")
print(f"Scale Factor: {scale_factor:.4f}")
print(f"Scaled Daily Volatility (EWMA): {scaled_daily_vol:0.2%}")
print(f"Scaled Annual Volatility (EWMA): {scaled_annual_vol:0.2%}")
print(f"Scaled Daily Portfolio Return: {scaled_avg_daily_return:0.2%}")
print(f"Scaled Annual Portfolio Return: {scaled_annual_return:0.2%}")

print("\nScaled Weights:")
for asset, w in zip(returns.columns, scaled_weights):
    print(f"{asset}: {w:.6f}")

# --- EXPORT SCALED WEIGHTS TO CSV ---
scaled_df = pd.DataFrame({
    "asset": returns.columns,
    "scaled_weight": scaled_weights
})

scaled_df.to_csv("ee_scaled_weights.csv", index=False)
print("\nScaled weights exported to ee_scaled_weights.csv")

Loaded weights (aligned):
[-8.09554158e-01  5.57991429e-01  7.75940439e-01  5.32511804e-03
 -3.87378792e-01  7.23058149e-01 -9.78302073e-01 -9.54323508e-02
  2.91013446e-01 -2.99002105e-01 -4.73879821e-01  3.02222424e-01
  3.40908207e-04 -2.63906553e-02  2.33423349e-01  9.59691366e-01
 -7.79066672e-01]

Asset order:
['AUDUSD', 'EURUSD', 'GBPUSD', 'NZDUSD', 'USDJPY', 'USDCHF', 'USDCAD', 'XAGUSD', 'XAUUSD', 'WTI', 'SPX500', 'US100', 'US30', 'DE30', 'JPN225', 'UK100', 'FRA40']

--- EWMA VOLATILITY FORECAST ---
EWMA Daily Volatility: 0.38%
EWMA Annual Volatility: 6.08%
EWMA Daily Portfolio Return: 0.03%
EWMA Annual Portfolio Return: 8.25%

--- SCALED PORTFOLIO (Target Vol = 10%) ---
Scale Factor: 1.6457
Scaled Daily Volatility (EWMA): 0.63%
Scaled Annual Volatility (EWMA): 10.00%
Scaled Daily Portfolio Return: 0.05%
Scaled Annual Portfolio Return: 13.58%

Scaled Weights:
AUDUSD: -1.332297
EURUSD: 0.918296
GBPUSD: 1.276978
NZDUSD: 0.008764
USDJPY: -0.637516
USDCHF: 1.189949
USDCAD: -1.61000

### Lot Sizing

In [ ]:
import math
import pandas as pd
import MetaTrader5 as mt5
import numpy as np

# ============================
# USER CONFIG
# ============================
EQUITY = 99438.93  # set your account equity

# FX-exempt symbols (no price in formula)
FX_EXEMPT = ["USDJPY", "USDCHF", "USDCAD"]

# Global index → FX mapping
INDEX_FX_MAP = {
    "DE30": "EURUSD",
    "JPN225": "USDJPY",
    "UK100": "GBPUSD",
    "FRA40": "EURUSD"
}

# ============================
# MT5 INITIALIZATION
# ============================
mt5.initialize()

def get_latest_price(symbol):
    tick = mt5.symbol_info_tick(symbol)
    if tick is None:
        return None
    return tick.ask

def fetch_prices(assets):
    return {a: get_latest_price(a) for a in assets}

def fetch_index_fx_rates():
    return {idx: get_latest_price(fx) for idx, fx in INDEX_FX_MAP.items()}

# ============================
# LOAD CSV FILES
# ============================
contract_df = pd.read_csv("ee_contract_size.csv")
weights_df = pd.read_csv("ee_scaled_weights.csv")

# Normalize column names
contract_df.columns = contract_df.columns.str.strip().str.lower()
weights_df.columns = weights_df.columns.str.strip().str.lower()

# Rename scaled_weight → weight
weights_df.rename(columns={"scaled_weight": "weight"}, inplace=True)

# Merge contract sizes + weights
df = contract_df.merge(weights_df, on="asset", how="left")

# ============================
# FETCH PRICES FROM MT5
# ============================
all_assets = df["asset"].tolist()
latest_prices = fetch_prices(all_assets)
index_fx_rates = fetch_index_fx_rates()

df["latest_price"] = df["asset"].map(latest_prices)

# ============================
# LOT SIZE CALCULATION
# ============================
def compute_lot(row):
    asset = row["asset"]
    weight = row["weight"]
    contract_size = row["contract_size"]
    price = row["latest_price"]

    # Missing weight or contract size
    if pd.isna(weight) or pd.isna(contract_size):
        return None

    # Contract size cannot be zero
    if contract_size == 0:
        return None

    # Rule 1: FX-exempt assets
    if asset in FX_EXEMPT:
        return (weight * EQUITY) / contract_size

    # Missing or zero price
    if price is None or price == 0:
        return None

    # Rule 2: Global index → convert to USD
    if asset in INDEX_FX_MAP:
        fx_rate = index_fx_rates.get(asset)
        if fx_rate is None or fx_rate == 0:
            return None
        # Invert FX rate for JP225
        if asset in ["JPN225"]:
            fx_rate = 1 / fx_rate

        price = price * fx_rate

        # After conversion, price still cannot be zero
        if price == 0:
            return None

    # Final safety check
    denominator = price * contract_size
    if denominator == 0:
        return None

    return (weight * EQUITY) / denominator

from decimal import Decimal, ROUND_HALF_UP

df["lot_size"] = df.apply(compute_lot, axis=1)

df["lot_size"] = df["lot_size"].apply(
    lambda x: Decimal(str(x)).quantize(Decimal("0.01"), rounding=ROUND_HALF_UP)
)




# ============================
# OUTPUT
# ============================
print(df[["asset", "weight", "contract_size", "latest_price", "lot_size"]].dropna())
df[["asset", "weight", "contract_size", "latest_price", "lot_size"]]\
    .dropna()\
    .to_csv("ee_lot_sizes_output.csv", index=False)

# ============================
# GROSS TOTAL LOT SIZE
# ============================
df["abs_lot_size"] = df["lot_size"].abs()
gross_total_lot_size = df["abs_lot_size"].sum()

print(f"\nGross Total Lot Size (absolute): {gross_total_lot_size:.4f}")
mt5.shutdown()

     asset    weight  contract_size  latest_price lot_size
0   AUDUSD -1.332297         100000       0.66867    -1.99
1   EURUSD  0.918296         100000       1.16306     0.79
2   GBPUSD  1.276978         100000       1.33988     0.95
3   NZDUSD  0.008764         100000       0.57306     0.02
4   USDJPY -0.637516         100000     157.95700    -0.64
5   USDCHF  1.189949         100000       0.80122     1.19
6   USDCAD -1.610008         100000       1.39181    -1.61
7   XAGUSD -0.157055           5000      79.98200    -0.04
8   XAUUSD  0.478926            100    4510.29000     0.11
9      WTI -0.492073            100      58.86000    -8.32
10  SPX500 -0.779872              1    6970.18000   -11.13
11   US100  0.497372              1   25783.60000     1.92
12    US30  0.000561              1   49524.50000     0.01
13    DE30 -0.043432              1   25347.80000    -0.15
14  JPN225  0.384149              1   53562.00000   112.66
15   UK100  1.579380              1   10159.90000    11.

True